## 爬取DCARD版位資訊

In [1]:
import json
import requests
import pymysql
import pandas as pd
import numpy as np

In [2]:
response = requests.get('https://www.dcard.tw/service/api/v2/forums').text
data = json.loads(response)
df = pd.DataFrame(data)
# df.head(3)

In [3]:
df = df[['id', 'alias', 'name', 'subscriptionCount', 'createdAt', 'updatedAt', 'isSchool', 'postCount']]
df.postCount = df.postCount.apply(lambda x : x['last30Days'])
df.createdAt = df.createdAt.apply(lambda x : x[:10])
df.updatedAt = df.updatedAt.apply(lambda x : x[:10])
df = df.sort_values(by = 'subscriptionCount', ascending=False)
df.head(10)

,id,alias,name,subscriptionCount,createdAt,updatedAt,isSchool,postCount
373,1ce3ebca-8701-42d5-b14c-076fc629bc8e,sex,西斯,627966,2020-02-04,2021-06-24,False,5431
228,42851318-b9e2-4a75-8a05-9fe180becefe,relationship,感情,571363,2020-02-04,2021-04-20,False,10680
224,be1a095b-175e-4523-9e06-66a05d939676,dressup,穿搭,561431,2020-02-04,2021-08-14,False,1632
217,cbd5285f-3cba-4bfc-86d0-1ab52d201459,makeup,美妝,470772,2020-02-04,2021-06-25,False,3356
233,06e07db4-c8b5-4dae-851f-e8a8236e770c,meme,梗圖,452353,2020-02-04,2020-08-31,False,2405
273,de977130-f3eb-467b-822e-b0e91124e9eb,food,美食,402602,2020-02-04,2021-04-20,False,1898
270,4c6964fc-8b39-4480-a844-847f09e4e09d,horoscopes,星座,357827,2020-02-04,2021-04-20,False,1964
230,255fd275-fec2-49d2-8e46-2e1557ffaeb0,talk,閒聊,355372,2020-02-04,2021-04-20,False,6275
346,fc7574ad-e1df-43d2-bb06-8825c718c065,trending,時事,341684,2020-02-04,2021-05-22,False,3301
340,3bceb81c-574e-4a96-8050-4df56096d0df,money,理財,311400,2020-02-04,2021-04-20,False,700


## 爬取最熱門版位文章資訊

# API 網址：https://www.dcard.tw/service/api/v2

## 說明	請求方法	路徑
## 全部文章	GET	/posts
## 看板資訊	GET	/forums
## 看板內文章列表	GET	/forums/{看板名稱}/posts
## 文章內文	GET	/posts/{文章ID}
## 文章內引用連結	GET	/posts/{文章ID}/links
## 文章內留言	GET	/posts/{文章ID}/comments

In [65]:
post_url = 'https://www.dcard.tw/service/api/v2/forums/boba/posts'
res = requests.get(post_url).text
data = json.loads(res)

In [6]:
len(data)

30

In [67]:
df_post = pd.DataFrame(data)
df_post = df_post[['id', 'title', 'excerpt', 'forumId', 'commentCount', 'likeCount', 'topics', 'school', 'gender', 'createdAt']]
df_post.columns = ['postid', 'title', 'excerpt', 'forumId', 'commentCount', 'likeCount', 'topics', 'author', 'gender', 'createdAt']
df_post


,postid,title,excerpt,forumId,commentCount,likeCount,topics,author,gender,createdAt
0,236794976,迷客夏的娜杯紅茶拿鐵如何？,今天新出的娜杯紅茶拿鐵有人去試了嗎~，還是有店員願意分享下味道如何~,fc52ee20-8732-41df-9751-49a8427187d6,14,10,"[迷客夏, 拿鐵, 美食, 飲料]",國立中正大學,F,2021-08-24T02:34:24.674Z
1,236793494,再睡五分鐘要進軍各大超商?!,剛剛看到官方突然發了這張圖 後面還是有紙盒的包裝，是不是要在超商販售了？？也太期待了吧 如果...,fc52ee20-8732-41df-9751-49a8427187d6,17,89,"[超商, 再睡五分鐘, 滴妹, 手搖杯, 飲料]",臺北城市科技大學,M,2021-08-23T17:10:24.298Z
2,236791888,嚐鮮分享-歇腳亭 下面一大包,上禮拜看到SHARE TEA 歇腳亭跟薩泰爾娛樂合作，一次集合喬瑟夫、博恩及賀瓏三位知名喜劇...,fc52ee20-8732-41df-9751-49a8427187d6,0,22,"[分享, 食記, 手搖飲]",醒吾科技大學,M,2021-08-23T13:57:30.329Z
3,236791559,#找工作 白巷子一個月打工心得,炎炎夏日，來一杯現打，現做的奶蓋綠茶太剛剛好不過了，台灣急忙的上班節奏，飲料就是你放下壓力的...,fc52ee20-8732-41df-9751-49a8427187d6,10,38,"[打工, 心得, 飲料, 白巷子, 工作]",哈利波霸,M,2021-08-23T13:17:07.115Z
4,236785021,珍奶,請問台南有沒有超級爆炸好喝的珍奶！，史上最好喝也可以啦吼🧋，拜託啦一定要推薦給我><，我怕我...,fc52ee20-8732-41df-9751-49a8427187d6,11,6,"[珍奶, 拜託]",南臺科技大學,F,2021-08-22T15:41:45.906Z
5,236784024,有人也是多料控嗎😳😳,一直以來，喝手搖飲都一定要加料，就算只是買杯綠茶也要加個珍珠，感覺嘴巴有咬到東西，才算真的有...,fc52ee20-8732-41df-9751-49a8427187d6,24,189,"[食記, 美食, 飲料, 手搖飲, 泱泱]",國立政治大學,F,2021-08-22T13:48:05.419Z
6,236782597,很適合告白的飲料名稱,【漫渡拾光－靜靜地喜歡你】這間起家於台南的飲料店，水果系列的飲料很強，真材實料，加好加滿！，...,fc52ee20-8732-41df-9751-49a8427187d6,4,88,"[飲料, 食記, 美食, 吃貨, 台南]",口袋朋友MikaMika,F,2021-08-22T10:33:18.812Z
7,236770211,［聯名］一手私藏世界紅茶×小王子👑,看到小王子聯名一手私藏...，就馬上跟我媽分享資訊（#言下之意：買給我喝（~‾▿‾）~ 不用...,fc52ee20-8732-41df-9751-49a8427187d6,15,63,"[聯名, 紅茶, 飲料, 手搖飲]",國立臺北大學,F,2021-08-20T15:43:28.459Z
8,236769348,自製鮮奶茶/飲料店鮮奶茶,不知道發在這正不正確！，不過想問一下板上的大家～有時候自己煮紅茶加鮮奶喝起來卻像奶+水一樣很...,fc52ee20-8732-41df-9751-49a8427187d6,17,22,[鮮奶茶],國立高雄應用科技大學,F,2021-08-20T13:59:30.306Z
9,236767678,#詢問 你最喜歡的飲料店？,請問大家，如果你們到別的縣市辦公或是出遊，最希望有哪家飲料店呢？ 因為學校作業需求，希望能有...,fc52ee20-8732-41df-9751-49a8427187d6,157,52,"[飲料, 喜歡, 討厭, 手搖飲, 飲料店]",大仁科技大學,M,2021-08-20T10:12:43.074Z


## 上傳Mysql資料庫

In [72]:
import sqlalchemy
from sqlalchemy import create_engine
database_username = 'jackyfu'
database_password = 'data8756'
database_ip       = 'sg2nlmysql29plsk.secureserver.net'
database_name     = 'Bigdata'

sql_engine = create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(database_username, database_password, database_ip, database_name))
df_post.astype(str).to_sql(name='dcard_posts', con=sql_engine, if_exists = 'replace', index=False)